In [ ]:
!pip install opencv-python

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import seaborn as sns
%matplotlib inline
import tensorflow as tf
import keras
from sklearn.metrics import f1_score
import cv2
from tqdm import tqdm
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D, Input
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.applications.resnet50 import ResNet50
from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:
datapath = "dataset/"
img_width, img_height = 224,224
classes = ["manipuri", "bharatanatyam", "odissi", "kathakali", "kathak", "sattriya", "kuchipudi", "mohiniyattam"]
class_names_label = {classes:i for i, classes in enumerate(set(classes))}
num_classes = len(classes)
# resnet_weights_path = 'resnet_weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
# resnet_weights_path = ('https://github.com/fchollet/deep-learning-models/'
#                        'releases/download/v0.2/'
#                        'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [3]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
def load_images(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (img_width, img_height),cv2.INTER_LINEAR)
    return resized

def norm_images(images):
    images = np.array(images,dtype = 'float32') / 255.  
    mean_img = np.mean(images,axos = 0)
    std_img = np.std(images,axis = 0)
    norm_img = (images - mean_img)/std_img
    return norm_img

def load_dataset(folder_name):
    images = []
    labels = []
    label_file = pd.read_csv(os.path.join(datapath,folder_name+".csv"))
    for file in tqdm(os.listdir(os.path.join(datapath,folder_name))):
        if(folder_name == "train"):
            label = class_names_label[label_file[label_file["Image"] == file]["target"].values[0]]
            labels.append(label)
        else:
            labels.append(file)
        image = load_images(os.path.join(datapath,folder_name,file))
        images.append(image)
    norm_img = np.array(images,dtype = 'float32') / 255.  
    return (norm_img,labels)

In [5]:
(x_train, y_train)=load_dataset("train")
x_train,y_train = shuffle(x_train,y_train,random_state = 42)
(x_test, img_name) = load_dataset("test")
y_train = np.array(y_train, dtype = 'int16')
y_train = to_categorical(y_train)

print(y_train.shape)

100%|██████████| 156/156 [00:00<00:00, 231.70it/s]

(364, 8)


In [ ]:
def generate_sol(name):
    predictions = model.predict(x_test)
    predictions = np.argmax(predictions,axis = 1)
    y_maps = dict()
    y_maps = {v:k for k,v in class_names_label.items()}
    print(y_maps)
    print(predictions)
    pred_labels = [y_maps[k] for k in predictions]
    print(pred_labels)
    sub = pd.DataFrame({'Image':img_name,'target':pred_labels})
    model.save('resnet_weights/'+name+'weights.h5')
    sub.to_csv(name+'sol.csv', index = False)

# Experiment 1

In [8]:
model = Sequential()
model.add(Convolution2D(32, (3,3), activation='relu', padding='same',input_shape = (img_width, img_height,3)))
model.add(Convolution2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(64, (3,3), activation='relu', padding='same'))
model.add(Convolution2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(128, (3,3), activation='relu', padding='same'))
model.add(Convolution2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 222, 222, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 111, 111, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 109, 109, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 64)       

In [9]:
early_stops = EarlyStopping(patience=3, monitor='val_accuracy')

model.fit(x_train, y_train, batch_size=100, epochs=10, validation_split=0.3, callbacks=[early_stops])
generate_sol("cnn")

Train on 254 samples, validate on 110 samples
Epoch 1/10
254/254 [==============================] - 23s 89ms/step - loss: 2.1289 - accuracy: 0.1260 - val_loss: 2.0737 - val_accuracy: 0.1636
Epoch 2/10
254/254 [==============================] - 19s 76ms/step - loss: 2.0751 - accuracy: 0.1772 - val_loss: 2.0751 - val_accuracy: 0.2091
Epoch 3/10
254/254 [==============================] - 27s 106ms/step - loss: 2.0715 - accuracy: 0.1811 - val_loss: 2.0602 - val_accuracy: 0.1545
Epoch 4/10
254/254 [==============================] - 27s 106ms/step - loss: 2.0562 - accuracy: 0.1929 - val_loss: 2.0634 - val_accuracy: 0.1727
Epoch 5/10
254/254 [==============================] - 28s 110ms/step - loss: 2.0472 - accuracy: 0.1811 - val_loss: 2.0247 - val_accuracy: 0.1364
{0: 'mohiniyattam', 1: 'bharatanatyam', 2: 'odissi', 3: 'sattriya', 4: 'kathakali', 5: 'kuchipudi', 6: 'manipuri', 7: 'kathak'}
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 2 4 2 4 4 4 4 4 4 4 4 4 4

# Transfer Learning With ResNet50 

In [16]:
model = Sequential()
model.add(ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet',input_tensor=Input(shape=(img_width,img_height,3))))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_5 (Dense)              (None, 128)               262272    
_________________________________________________________________
dense_6 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 2056      
Total params: 23,885,064
Trainable params: 23,831,944
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
early_stops = EarlyStopping(patience=3, monitor='val_accuracy')

model.fit(x_train, y_train, batch_size=100, epochs=15, validation_split=0.3, callbacks=[early_stops])
generate_sol("pretrained")

Train on 254 samples, validate on 110 samples
Epoch 1/15
254/254 [==============================] - 144s 567ms/step - loss: 1.9259 - accuracy: 0.2874 - val_loss: 2.2863 - val_accuracy: 0.1000
Epoch 2/15
200/254 [======================>.......] - ETA: 21s - loss: 0.9830 - accuracy: 0.7000 

# ResNet50 without data augumentation

In [ ]:
resnet = ResNet50(include_top=False, weights='imagenet', 
                                     input_shape=(img_width, img_height,3))

output = resnet.layers[-1].output
output = Flatten()(output)

resnet = Model(resnet.input, output)
resnet.trainable = False

for layer in resnet.layers:
    layer.trainable = False

resnet.summary()

In [ ]:
pd.set_option('max_colwidth', -1)

layers = [(layer, layer.name, layer.trainable) for layer in resnet.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

In [ ]:
print("Trainable layers:", resnet.trainable_weights)

In [ ]:
model = Sequential()
model.add(resnet)
model.add(Dense(512, activation='relu', input_dim=(img_width, img_height,3)))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

In [ ]:
early_stops = EarlyStopping(patience=3, monitor='val_accuracy')

history1 = model.fit(x_train, y_train, batch_size=100, epochs=15, validation_split=0.3, callbacks=[early_stops])
generate_sol("withoutdataaug")

# ResNet with Data Augumentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 40, width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.3)
#val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow(x_train, y_train,shuffle = True,subset="training",batch_size=32)
val_generator = datagen.flow(x_train, y_train,shuffle = True,subset = "validation", batch_size=32)
#val_generator = val_datagen.flow(validation_imgs, validation_labels_enc, batch_size=30)

In [ ]:
model = Sequential()
model.add(resnet)
model.add(Dense(512, activation='relu', input_dim=(img_width, img_height,3)))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch=100, epochs=15,
                          validation_data=val_generator, validation_steps=50, verbose=1)

In [ ]:
model.save('resnet_weights/tlearn_transfer_cnn.h5')

In [ ]:
# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
# t = f.suptitle('Pre-trained CNN (Transfer Learning) Performance', fontsize=12)
# f.subplots_adjust(top=0.85, wspace=0.3)

# epoch_list = list(range(1,11))
# ax1.plot(epoch_list, history.history['accuracy'], label='Train Accuracy')
# ax1.plot(epoch_list, history.history['val_accuracy'], label='Validation Accuracy')
# ax1.set_xticks(np.arange(0, 31, 5))
# ax1.set_ylabel('Accuracy Value')
# ax1.set_xlabel('Epoch')
# ax1.set_title('Accuracy')
# l1 = ax1.legend(loc="best")

# ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
# ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
# # ax2.set_xticks(np.arange(0, 31, 5))
# ax2.set_ylabel('Loss Value')
# ax2.set_xlabel('Epoch')
# ax2.set_title('Loss')
# l2 = ax2.legend(loc="best")